
**Segmenting and clustering neighborhoods in the city of Toronto**

Week3 Peer Graded Assignment

Data Science IBM Capstone Project

**Part 3**

In [1]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

     |████████████████████████████████| 102kB 4.0MB/s 
Packages installed.


In [2]:
pip install BeautifulSoup4

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [5]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [6]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [7]:
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,T1A,T2A,T3A,43.64869,-79.38544
1,T4A,T5A,T6A,43.64869,-79.38544
2,T7A,Drayton Valley,T8A,43.64869,-79.38544
3,T9A,Wetaskiwin,T1B,43.64869,-79.38544
4,T2B,T3B,T4B,43.64869,-79.38544
...,...,...,...,...,...
253,T0L,Kananaskis Improvement District,T0M,43.64869,-79.38544
254,T0N,Not in use,T0P,43.64869,-79.38544
255,T0R,Not in use,T0S,43.64869,-79.38544
256,T0T,Not in use,T0V,43.64869,-79.38544


In [8]:
neighbors.shape

(258, 5)

In [9]:

VERSION = '20180605' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     43.67635739999999, 
#     79.2930312, 
#     radius, 
#     LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore?&client_id=JGGBRN5XODTLZGJOMCSWIQMRH1JLGJKPSFR10XNB2R5U25GR&client_secret=KWRAMLK2HOJBQ2XLICLKXRU3M4HOCC1U2VG4Y4OPP5JF03QX&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

results = requests.get(url).json()
results

Latitude and longitude values of T3A are 43.648690000000045, -79.38543999999996.


{'meta': {'code': 200, 'requestId': '608041224f07cc19e67c371a'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bd461bc77b29c74a07d9282-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d159941735',
         'name': 'Trail',
         'pluralName': 'Trails',
         'primary': True,
         'shortName': 'Trail'}],
       'id': '4bd461bc77b29c74a07d9282',
       'location': {'address': 'Glen Manor',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Queen St.',
        'distance': 89,
        'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


In [12]:

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [13]:

def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

T3A
T6A
T8A
T1B
T4B
T7B
Not assigned
T3C
T5C
Not assigned
Vegreville
T4E
T7E
T1G
T4G
T7G
Devon
T3H
T5H
Not assigned
T1J
T4J
T7J
T2K
Not assigned
T7K
Banff
T4L
T7L
T9L
T2M
Blackfalds
T7M
T1N
T4N
T7N
T9N
T2P
T5P
Westlock
T2R
T5R
Not assigned
T9R
T2S
Sylvan Lake
T7S
T9S
Tsuut'ina
T5T
Not assigned
T9T
T2V
T5V
Hinton
Lloydminster
T2W
T7W
Wainwright
T2X
T5X
T8X
T1Y
Not assigned
T7Y
T2Z
T4Z
Not assigned
T8Z
Central Medicine Hat

Penbrooke Meadows, Marlborough

Dalhousie, Edgemont, Hamptons, Hidden Valley

East Airdrie

West Clareview, East Londonderry

North Capilano

Drayton Valley

West Sherwood Park

Wetaskiwin

South Medicine Hat

Forest Lawn, Dover, Erin Woods

Montgomery, Bowness, Silver Springs, Greenwood

West Airdrie

East North Central, West Beverly

SE Capilano, West Southeast Industrial, East Bonnie Doon

Not assigned

Outer Southwest

Not assigned

North Medicine Hat

Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains

Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta



In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(1290, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,T3A,43.64869,-79.38544,Glen Manor Ravine,43.676821,-79.293942,Trail
1,T3A,43.64869,-79.38544,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,T3A,43.64869,-79.38544,Grover Pub and Grub,43.679181,-79.297215,Pub
3,T3A,43.64869,-79.38544,Upper Beaches,43.680563,-79.292869,Neighborhood
4,T3A,43.64869,-79.38544,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [16]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Athabasca\n,5,5,5,5,5,5
Banff,5,5,5,5,5,5
Banff\n,5,5,5,5,5,5
Barrhead\n,5,5,5,5,5,5
Beaumont\n,5,5,5,5,5,5
...,...,...,...,...,...,...
West and South Lethbridge\n,5,5,5,5,5,5
Westlock,5,5,5,5,5,5
Westlock\n,5,5,5,5,5,5


In [17]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 5 uniques categories.


In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Trail,Coffee Shop,Health Food Store,Neighborhood,Pub
0,1,0,0,T3A,0
1,0,0,1,T3A,0
2,0,0,0,T3A,1
3,0,0,0,T3A,0
4,0,1,0,T3A,0


In [19]:

# examine the new dataframe size.

toronto_onehot.shape

(1290, 5)

In [20]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Trail,Coffee Shop,Health Food Store,Pub
0,Athabasca\n,0.2,0.2,0.2,0.2
1,Banff,0.2,0.2,0.2,0.2
2,Banff\n,0.2,0.2,0.2,0.2
3,Barrhead\n,0.2,0.2,0.2,0.2
4,Beaumont\n,0.2,0.2,0.2,0.2
...,...,...,...,...,...
209,West and South Lethbridge\n,0.2,0.2,0.2,0.2
210,Westlock,0.2,0.2,0.2,0.2
211,Westlock\n,0.2,0.2,0.2,0.2
212,Wetaskiwin\n,0.2,0.2,0.2,0.2


In [21]:
# confirm the new size

toronto_grouped.shape

(214, 5)

In [22]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Athabasca
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Banff----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Banff
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Barrhead
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Beaumont
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Blackfalds----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Blackfalds
----
               venue  freq
0              Trail   0.2
1

In [23]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Athabasca\n,Pub,Health Food Store,Coffee Shop
1,Banff,Pub,Health Food Store,Coffee Shop
2,Banff\n,Pub,Health Food Store,Coffee Shop
3,Barrhead\n,Pub,Health Food Store,Coffee Shop
4,Beaumont\n,Pub,Health Food Store,Coffee Shop


In [25]:

# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  # Remove the CWD from sys.path while we load stuff.


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,T1A,T2A,T3A,43.64869,-79.38544,0,Pub,Health Food Store,Coffee Shop
1,T4A,T5A,T6A,43.64869,-79.38544,0,Pub,Health Food Store,Coffee Shop
2,T7A,Drayton Valley,T8A,43.64869,-79.38544,0,Pub,Health Food Store,Coffee Shop
3,T9A,Wetaskiwin,T1B,43.64869,-79.38544,0,Pub,Health Food Store,Coffee Shop
4,T2B,T3B,T4B,43.64869,-79.38544,0,Pub,Health Food Store,Coffee Shop


In [27]:
neighbors.shape

(258, 5)